In [1]:
# def get_file_counts(filename):
#     with open(filename) as fp:
#         count = 0
#         for _ in fp:
#             count += 1
#     return count

file_name_list = ['2015_32384541_32M-006.csv', '2016_31758353_31M-007.csv', '2017_24987298_24M-004.csv',\
                  '2018_20731058_20M-005.csv', '2019_16476247_16M-003.csv', '2020_3888214_3M.csv', '2021_2576772_2M.csv']


# file_counts = []

# for i in file_name_list:
#     file_counts.append(get_file_counts(i))

# print(file_counts)

file_counts = [32384542, 31758354, 24987299, 20731059, 16476248, 3888215, 2576773]
total_size = sum(file_counts)
print(total_size)

132802490


In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
import random

In [3]:
def my_generator(chunks, file_list):
    while True:
        for f in file_list:
            for data in pd.read_csv(f, chunksize=chunks):
                data = data[data['Trip Seconds'].notna()]
                data = data[data['Trip Miles'].notna()]
                data = data[data['Fare'].notna()]
                data = data[data['Trip Start Timestamp'].notna()]
                data['month'] = pd.to_datetime(data['Trip Start Timestamp']).dt.month
                x = data[['Trip Seconds', 'Trip Miles', 'month']].to_numpy()
                y = data['Fare'].to_numpy()
                yield x, y
            


def get_validation_data(validation_size, file_size, file_list, p):
    if p == 0:
        p = validation_size/file_size
    li = []
    for f in file_list:
        df = pd.read_csv(f, header=0, skiprows=lambda i: i>0 and random.random() > p)
        li.append(df)
    if len(li) > 1:
        data = pd.concat(li, axis=0, ignore_index=True)
    else:
        data = li[0]
    data = data[data['Trip Seconds'].notna()]
    data = data[data['Trip Miles'].notna()]
    data = data[data['Fare'].notna()]
    data = data[data['Trip Start Timestamp'].notna()]
    data['month'] = pd.to_datetime(data['Trip Start Timestamp']).dt.month
    x = data[['Trip Seconds', 'Trip Miles', 'month']].to_numpy()
    y = data['Fare'].to_numpy()
    return x, y
    

# Large Model

In [4]:
# NN_model = Sequential()

# # The Input Layer :
# NN_model.add(Dense(16, kernel_initializer='normal',input_dim = 3, activation='relu'))

# # The Hidden Layer :
# NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))
# NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))

# # The Output Layer :
# NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


In [5]:
# file_length = file_counts[2] + file_counts[3]
# file_list = [file_name_list[2], file_name_list[3]]
# chunksize = 15000

# NN_model.fit_generator(my_generator(chunksize, file_list),
#           steps_per_epoch=file_length//chunksize,
#           epochs=3,
#           verbose=2,
#           validation_data=validator_generator(chunksize),
#           validation_steps=16476248//chunksize
#             )

# 2021 Model

In [6]:
NN_model_2021 = Sequential()

# The Input Layer :
NN_model_2021.add(Dense(16, kernel_initializer='normal',input_dim = 3, activation='relu'))

# The Hidden Layer :
NN_model_2021.add(Dense(32, kernel_initializer='normal',activation='relu'))
NN_model_2021.add(Dense(32, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model_2021.add(Dense(1, kernel_initializer='normal',activation='linear'))

NN_model_2021.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


In [10]:
file_length = file_counts[6]
file_list = [file_name_list[6]]
chunksize = 15000
validation_set = get_validation_data(100000, file_length, file_list, 0)

NN_model_2021.fit_generator(my_generator(chunksize, file_list),
          steps_per_epoch=file_length//chunksize,
          epochs=10,
          verbose=2,
          validation_data=validation_set, 
            )

C:\Users\Pulak\AppData\Local\Temp/ipykernel_8688/3908493545.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  NN_model_2021.fit_generator(my_generator(chunksize, file_list),


Epoch 1/10
171/171 - 16s - loss: 4.1086 - mean_absolute_error: 4.1086 - val_loss: 4.0301 - val_mean_absolute_error: 4.0301 - 16s/epoch - 92ms/step
Epoch 2/10
171/171 - 16s - loss: 3.8082 - mean_absolute_error: 3.8082 - val_loss: 3.7527 - val_mean_absolute_error: 3.7527 - 16s/epoch - 93ms/step
Epoch 3/10
171/171 - 16s - loss: 3.7696 - mean_absolute_error: 3.7696 - val_loss: 3.4057 - val_mean_absolute_error: 3.4057 - 16s/epoch - 93ms/step
Epoch 4/10
171/171 - 16s - loss: 3.7234 - mean_absolute_error: 3.7234 - val_loss: 3.3707 - val_mean_absolute_error: 3.3707 - 16s/epoch - 93ms/step
Epoch 5/10
171/171 - 16s - loss: 3.7150 - mean_absolute_error: 3.7150 - val_loss: 3.5881 - val_mean_absolute_error: 3.5881 - 16s/epoch - 93ms/step
Epoch 6/10
171/171 - 16s - loss: 3.7114 - mean_absolute_error: 3.7114 - val_loss: 3.4846 - val_mean_absolute_error: 3.4846 - 16s/epoch - 93ms/step
Epoch 7/10
171/171 - 16s - loss: 3.6350 - mean_absolute_error: 3.6350 - val_loss: 3.2623 - val_mean_absolute_error: 3.

In [15]:
NN_model_2021.save('NN_model_2021')

INFO:tensorflow:Assets written to: NN_model_2021\assets


# 2015 Model

In [12]:
NN_model_2015 = Sequential()

# The Input Layer :
NN_model_2015.add(Dense(16, kernel_initializer='normal',input_dim = 3, activation='relu'))

# The Hidden Layer :
NN_model_2015.add(Dense(32, kernel_initializer='normal',activation='relu'))
NN_model_2015.add(Dense(32, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model_2015.add(Dense(1, kernel_initializer='normal',activation='linear'))

NN_model_2015.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])


In [13]:
file_length = file_counts[0]
file_list = [file_name_list[0]]
chunksize = 15000
validation_set = get_validation_data(100000, file_length, file_list, 0)

NN_model_2015.fit_generator(my_generator(chunksize, file_list),
          steps_per_epoch=file_length//chunksize,
          epochs=3,
          verbose=2,
          validation_data=validation_set, 
            )

C:\Users\Pulak\AppData\Local\Temp/ipykernel_8688/2611377337.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  NN_model_2015.fit_generator(my_generator(chunksize, file_list),


Epoch 1/3
2158/2158 - 1889s - loss: 2.8509 - mean_absolute_error: 2.8509 - val_loss: 2.3865 - val_mean_absolute_error: 2.3865 - 1889s/epoch - 875ms/step
Epoch 2/3
2158/2158 - 1895s - loss: 2.4085 - mean_absolute_error: 2.4085 - val_loss: 3.9199 - val_mean_absolute_error: 3.9199 - 1895s/epoch - 878ms/step
Epoch 3/3
2158/2158 - 1884s - loss: 2.3843 - mean_absolute_error: 2.3843 - val_loss: 3.5050 - val_mean_absolute_error: 3.5050 - 1884s/epoch - 873ms/step


In [14]:
NN_model_2015.save('NN_model_2015')

INFO:tensorflow:Assets written to: NN_model_2015\assets


In [8]:
def predict_fare(model, time, distance, month):
    return model.predict([[time, distance, month]])[0][0]


# predict_fare(1000, 0.5, 7)

In [9]:
# NN_model.save('NN_model')